In [1]:
import pandas as pd
import os
import re
import numpy as np

In [ ]:
all_dfs = []


ignore_words = ['Código', 'Ano/Período', 'Nome','Funcional','cpf','Titulacao','Titulação','Nome.1']


for file in os.listdir():
    
    if file.startswith('~$'):
        continue
    if file.endswith('.xlsx') or file.endswith('.xls'):
        
        file_path = os.path.abspath(file)
        
        xls = pd.ExcelFile(file_path)
        
        for sheet_name in xls.sheet_names:
            
            first_line = pd.read_excel(file_path, sheet_name=sheet_name, nrows=1)
           
            skiprows = 1 if first_line.applymap(np.isreal).any().any() else 0
           
            df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)
            
            df = df[~df.apply(lambda row: row.astype(str).str.contains('|'.join(ignore_words)).any(), axis=1)]
           
            df['SheetName'] = sheet_name
            
            if not sheet_name.isnumeric():
                
                numbers_in_title = re.findall(r'\d+', file)
                if numbers_in_title:
                    df['NumberInTitle'] = numbers_in_title[0]
            
            all_dfs.append(df)

In [3]:

geral_df = pd.concat(all_dfs)

In [ ]:
geral_df.head(3)

In [5]:
rename_columns = {'Disciplina': 'cod_disc', 'Unnamed: 1': 'ano/per','Coordenador/professor':'Nome','Unnamed: 3':'Funcional','Unnamed: 4':'cpf','Unnamed: 5':'Titulacao'}


geral_df = geral_df.rename(columns=rename_columns)

In [ ]:
geral_df.head()

In [7]:
geral_df.to_excel('aquivo_final.xlsx',index = False)